# 2022 Shopify Challenge

In [105]:
# importing the required libraries
import pandas as pd
import hvplot.pandas

# loading the dataset into a pandas dataframe

In [127]:
# using a pd data frame to load the data and begin to explore the data
data = pd.read_csv('RawData/data.csv' , infer_datetime_format=True, parse_dates=['created_at'])
# checking the data types of the columns
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        5000 non-null   int64         
 1   shop_id         5000 non-null   int64         
 2   user_id         5000 non-null   int64         
 3   order_amount    5000 non-null   int64         
 4   total_items     5000 non-null   int64         
 5   payment_method  5000 non-null   object        
 6   created_at      5000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 273.6+ KB


In [136]:
# need to convert payments methods to the correct format
data['payment_method'] = data['payment_method'].astype('string')
display(data.info())
display(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        5000 non-null   int64         
 1   shop_id         5000 non-null   int64         
 2   user_id         5000 non-null   int64         
 3   order_amount    5000 non-null   int64         
 4   total_items     5000 non-null   int64         
 5   payment_method  5000 non-null   string        
 6   created_at      5000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(5), string(1)
memory usage: 273.6 KB


None

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 04:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 04:35:11


In [135]:
# output some statistics about the data
data.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,2500.500000,50.078800,849.092400,3145.128000,8.78720
std,1443.520003,29.006118,87.798982,41282.539349,116.32032
min,1.000000,1.000000,607.000000,90.000000,1.00000
25%,1250.750000,24.000000,775.000000,163.000000,1.00000
50%,2500.500000,50.000000,849.000000,284.000000,2.00000
75%,3750.250000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,704000.000000,2000.00000


Since there was describe function output above we clearly see that the order amount has some outlier data causing the mean to be higher than expected with values of **704000** for the order amount so lets investigate the data further.

Lets explore the data further by plotting the distibution of the order amount.

In [137]:
data.hvplot.scatter(x='order_id' , y = 'order_amount') 

:Scatter   [order_id]   (order_amount)

if we examine the data we can see that the order amount has some points that are far above the the avarage order amount of $3145.13 and thus its not only the 704000 value in question. 
if we zoom in we can see multiple orders of 25725 also so we will focus the order amount above 25000 for the further analysis below 

In [166]:
# created a new data frame to store the data for the above 25000 order amount to investigate further based on the shop
temp_for_above_25k = data.loc[data['order_amount'] > 25000] 
duplicated_for_25k = temp_for_above_25k['created_at'].duplicated()
result_for_above_25k = pd.concat([temp_for_above_25k, duplicated_for_25k], axis=1 , names=['created_at', 'duplicated'])
result_for_above_25k.to_csv('RawData/result_for_above_25K.csv')
temp_for_above_25k.head()

# charting a histogram of the data to investigate the distribution of the data by shop and users 
chart_by_shop =temp_for_above_25k.hvplot.hist( 
                y='order_amount', 
                by = 'shop_id' ,
                title='Orders Above 25K By Shop' , 
                rot=45 , xlabel='Order Amount in $' , 
                ylabel='Frequency' , 
                height=500, 
                width=500 ) 
chart_by_user =temp_for_above_25k.hvplot.hist( 
                y='order_amount', 
                by = 'user_id' ,
                title='Orders Above 25K By User' , 
                rot=45 , xlabel='Order Amount in $' , 
                ylabel='Frequency' ,
                height=500, 
                width=500) 

chart_by_shop+chart_by_user


:Layout
   .NdOverlay.I  :NdOverlay   [shop_id]
      :Histogram   [order_amount]   (order_amount_count)
   .NdOverlay.II :NdOverlay   [user_id]
      :Histogram   [order_amount]   (order_amount_count)

Lets now Create the Same plot but for all prices above normal based on our 